# Milestone 1: Evaluation metrics

In [2]:
import pandas as pd

In [8]:
housing_df = pd.read_csv("data/2023_Property_Tax_Assessment.csv")
housing_df.head()

,X,Y,OBJECTID,roll,address,bldg,meters,feet,garage,firepl,bsmt,bdevl,assess_2022,latitude,longitude
0,65627.5814,5.915220e+06,1,133300004,21241 TWP RD 510,Split Level,150.590,1621,Y,Y,Y,N,382460,53.366050,-113.014015
1,76432.7311,5.916605e+06,2,1002300000,51055 RNG RD 202,1 Storey Basementless,123.560,1330,N,Y,N,N,280370,53.377049,-112.851377
2,76990.7865,5.916759e+06,3,1002301008,20156 HWY 14,1 Storey Slab on Grade,104.980,1130,N,N,N,N,402000,53.378350,-112.842955
3,76405.6156,5.916178e+06,4,1002700001,51037 RNG RD 202,Manufactured Home,66.611,717,N,N,N,N,3690,53.373211,-112.851888
4,77040.0860,5.915515e+06,5,1002701009,20148 TWP RD 510,1 1/2 Storey & Basement,123.830,1333,Y,Y,Y,Y,295910,53.367166,-112.842518
